# Investigate move/word and left/right task decomposition

Train classifiers to discriminate between move/word and left right.

To goal is to see which classifiers  fit the data well and 
how suitable is this decomposition.

In [1]:
import os
import math

import numpy as np
import pandas as pd


from sklearn.utils import shuffle


import pdb

from readers_preprocess import read_filter
from time_to_freq_domain import eval_psd, eval_psd_not_modulated, fft_signal, eval_band_power

from dim_reduction import  rank_features, select_best_chanels
from transformers import flatten_data, transform_to_one_chanel_data
from classification import train_test
from sklearn.decomposition import PCA


Train ML algorithams on the 1-st session and evaluate then on the 2-nd for mulicalss classification.

The 3-rd session is not used here.

In [3]:


all_paths = [['data_bci\\row_data\\subject1\\'], ['data_bci\\row_data\\subject2\\'],['data_bci\\row_data\\subject3\\']]
columns_to_read =  ['Fp1', 'AF3' ,'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5',
                   'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6',
                   'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz','class']
chanels_rank =  [25,  4,  9, 26, 30, 11,  7]

test_subject = '02'#test session 
train_subjects = ['01']# train session

subject_results = []
knn_results = []


#freq settings
num_perseg = 128
num_overlap = 64
freq=512
min_freq = 8
max_freq = 45
classification_report = []

for path in all_paths:
    cutoff_beggining = 0
    seq_len = 0
    cut_step = 0
    train_data, _, train_anots, test_data, _, test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
    seq_len = freq
    cut_step = int(seq_len/2)
    window_train_data, _, window_train_anots, window_test_data, _, window_test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
   

    y_train = train_anots
    y_test = window_test_annoations
        
    X_train =  eval_psd_not_modulated(train_data, num_perseg, num_overlap, freq, min_freq, max_freq)
    X_test =  eval_psd_not_modulated(window_test_data, num_perseg, num_overlap, freq, min_freq, max_freq)
        
    X_train = flatten_data(X_train[:,:,chanels_rank])
    X_test = flatten_data(X_test[:,:,chanels_rank]) 
        
    most_important_features = rank_features(X_train, y_train)
        
    X_train = X_train[:, most_important_features]
    X_test = X_test[:, most_important_features]
        
    pca = PCA(n_components=0.985)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    
    max_el = np.abs(X_train).max()
    X_train = X_train/max_el
    X_test = X_test/max_el

    
    report, algorithams_names = train_test(X_train, X_test, y_train, y_test)
    classification_report.append(report)


RBF SVM 
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.12, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.5859030837
linear SVM 
LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.559471365639
KNN Euclidian
0.614537444934
DT
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.605726872247
RBF SVM 
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1

In [4]:
alghortihams_results = pd.DataFrame(classification_report, columns=algorithams_names).T

alghortihams_results.columns = ['person_1_accuracy', 'person_2_accuracy', 'person_3_accuracy']
alghortihams_results.index.name = 'algoritham'

means = alghortihams_results.mean(axis=1)
stds = alghortihams_results.std(axis=1)

alghortihams_results['mean_accuracy'] = means
alghortihams_results['std_accuracy'] = stds
alghortihams_results = alghortihams_results.round(3)
alghortihams_results.to_excel("Multiclass_selection_report.xlsx")
alghortihams_results

,person_1_accuracy,person_2_accuracy,person_3_accuracy,mean_accuracy,std_accuracy
algoritham,,,,,
RBF SVM,0.586,0.425,0.511,0.507,0.081
linear SVM,0.559,0.427,0.524,0.504,0.069
KNN Euclidian,0.615,0.423,0.518,0.518,0.096
Decision tree,0.606,0.376,0.507,0.496,0.115


Binary classifyer algorithams 

In [5]:
all_paths = [['data_bci\\row_data\\subject1\\'], ['data_bci\\row_data\\subject2\\'],['data_bci\\row_data\\subject3\\']]
columns_to_read =  ['Fp1', 'AF3' ,'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5',
                   'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6',
                   'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz','class']

test_subject = '02'
freq = 512
train_subjects = ['01']

min_freq = 8



num_perseg = 128
num_overlap = 64
freq=512
min_freq = 8
max_freq = 45

report_move_vs_word = []
report_left_vs_right = []


for path in all_paths:
    cutoff_beggining = 0
    seq_len = 0
    cut_step = 0
    train_data, _, train_anots, test_data, _, test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
    seq_len = freq
    cut_step = int(seq_len/2)
    window_train_data, _, window_train_anots, window_test_data, _, window_test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
   

    y_train = train_anots
    y_train_move_word = (y_train < 2)
    y_train_left_right = y_train[y_train!=2]
    
    
    y_test = window_test_annoations
    y_test_move_word = (y_test < 2)
    y_test_left_right = y_test[y_test!=2]
    
    
    X_train =  eval_psd_not_modulated(train_data, num_perseg, num_overlap, freq, min_freq, max_freq)
    X_test =  eval_psd_not_modulated(window_test_data, num_perseg, num_overlap, freq, min_freq, max_freq)
        
    X_train = flatten_data(X_train[:,:,chanels_rank])
    X_test = flatten_data(X_test[:,:,chanels_rank]) 
    
    
    features_move_word = rank_features(X_train, y_train_move_word)
    X_train_move_word = X_train[:, features_move_word]
    X_test_move_word = X_test[:, features_move_word]   
    
    pca = PCA(n_components=0.985)
    X_train_move_word = pca.fit_transform(X_train_move_word)
    X_test_move_word = pca.transform(X_test_move_word)
    
    
    X_train_left_right = X_train[y_train != 2]
    X_test_left_right = X_test[y_test != 2]
    
    features_left_right = rank_features(X_train_left_right, y_train_left_right)
    
    X_train_left_right = X_train_left_right[:, features_left_right]
    X_test_left_right = X_test_left_right[:, features_left_right]
        

    pca = PCA(n_components=0.985)
    X_train_left_right = pca.fit_transform(X_train_left_right)
    X_test_left_right = pca.transform(X_test_left_right)

    max_el = X_train_move_word.max() 
    min_el = X_train_move_word.min()

    X_train_move_word = (X_train_move_word-min_el)/(max_el-min_el)
    X_test_move_word = (X_test_move_word-min_el)/(max_el-min_el)
    
    
    
    max_el = X_train_left_right.max() 
    min_el = X_train_left_right.min()
    
    X_train_left_right = (X_train_left_right-min_el)/(max_el-min_el)
    X_test_left_right = (X_test_left_right-min_el)/(max_el-min_el)
    
    
    report, algorithams_names = train_test(X_train_move_word, X_test_move_word, y_train_move_word, y_test_move_word)
    report_move_vs_word.append(report)
    
    report, algorithams_names = train_test(X_train_left_right, X_test_left_right, y_train_left_right, y_test_left_right)
    report_left_vs_right.append(report)

RBF SVM 
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.12, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.770925110132
linear SVM 
LinearSVC(C=100, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.76872246696
KNN Euclidian
0.777533039648
DT
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.773127753304
RBF SVM 
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0

In [6]:
df_move_vs_word = pd.DataFrame(report_move_vs_word, columns=algorithams_names).T
df_move_vs_word.columns = ['person_1_accuracy', 'person_2_accuracy', 'person_3_accuracy']
df_move_vs_word.index.name = 'algoritham'

m = df_move_vs_word.mean(axis=1)
s = df_move_vs_word.std(axis=1)

df_move_vs_word['mean_accuracy'] = m
df_move_vs_word['std_accuracy'] = s

df_move_vs_word.sort_values(by='mean_accuracy', ascending=False)
df_move_vs_word = df_move_vs_word.round(3)
df_move_vs_word.to_excel("Test_Move_word_report_custom_feats.xlsx")

df_move_vs_word

,person_1_accuracy,person_2_accuracy,person_3_accuracy,mean_accuracy,std_accuracy
algoritham,,,,,
RBF SVM,0.771,0.743,0.753,0.756,0.014
linear SVM,0.769,0.750,0.758,0.759,0.009
KNN Euclidian,0.778,0.741,0.758,0.759,0.018
Decision tree,0.773,0.710,0.656,0.713,0.059


In [8]:
df_left_vs_right = pd.DataFrame(report_left_vs_right, columns=algorithams_names).T
df_left_vs_right.columns = ['person_1_accuracy', 'person_2_accuracy', 'person_3_accuracy']
df_left_vs_right.index.name = 'algoritham'

m = df_left_vs_right.mean(axis=1)
s = df_left_vs_right.std(axis=1)

df_left_vs_right['mean_accuracy'] = m
df_left_vs_right['std_accuracy'] = s
df_left_vs_right.sort_values(by='mean_accuracy', ascending=False)
df_left_vs_right = df_left_vs_right.round(3)
df_left_vs_right.to_excel("Test_letf_right_report_custom_feats.xlsx")

df_left_vs_right

,person_1_accuracy,person_2_accuracy,person_3_accuracy,mean_accuracy,std_accuracy
algoritham,,,,,
RBF SVM,0.641,0.337,0.618,0.532,0.170
linear SVM,0.597,0.350,0.574,0.507,0.137
KNN Euclidian,0.652,0.353,0.557,0.521,0.153
Decision tree,0.634,0.422,0.564,0.540,0.108
